In [1]:
from rosemary import jpt_parse_args, jpt_setup; jpt_setup()

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [2]:
import argparse
import glob
import json
import os
import random
from collections import defaultdict
import torch

from tqdm import tqdm, trange
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from eval.utils import (
    generate_completions,
    load_hf_lm_and_tokenizer,
    query_openai_chat_model,
    dynamic_import_function,
)

from eval.toxigen.run_eval import score_generations

[2023-09-24 19:56:45,624] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("--data_dir", type=str, default="data/eval/toxigen")
parser.add_argument("--save_dir", type=str, default="results/toxigen")
parser.add_argument("--model_name_or_path", type=str, default=None, help="if specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="if specified, we will load the tokenizer from here.")
parser.add_argument("--use_slow_tokenizer", action="store_true", help="If given, we will use the slow tokenizer.")
parser.add_argument("--openai_engine", type=str, default=None, help="if specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="batch size for evaluation.")
parser.add_argument("--classifier_batch_size", type=int, default=32, help="batch size to use for toxicity classifier.")
parser.add_argument("--classifier_device", type=str, default="cuda", help="device to use for toxicity classifier.")
parser.add_argument("--load_in_8bit", action="store_true", help="load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, we will use the chat format for the prompts.")
parser.add_argument("--chat_formatting_function", type=str, default="eval.templates.create_prompt_with_tulu_chat_format", help="The function to use to create the chat format. This function will be dynamically imported. Please see examples in `eval/templates.py`.")
parser.add_argument("--use_vllm", action="store_true", help="If given, we will use vLLM to generate the predictions - much faster.")
parser.add_argument("--max_prompts_per_group", type=int, default=500, help="If given, we will only use this many prompts per group. Default to 500 (half the available prompts).")
parser.add_argument("--max_new_tokens", type=int, default=512)

model_name_or_path = '../results/baselines/huggyllama/llama-7b/'

cmd = f"""
    --data_dir ../data/eval/toxigen/ \
    --model_name_or_path {model_name_or_path} \
    --save_dir {model_name_or_path}/eval/toxigen/ \
    --eval_batch_size 10 \
    --use_chat_format
"""

args = jpt_parse_args(parser, cmd)


# model_name_or_path and openai_engine cannot be both None or both not None.
assert (args.model_name_or_path is None) != (
    args.openai_engine is None
), "Either model_name_or_path or openai_engine should be specified."
# (4*60+25)/80=3.31
# 110/150=.7
args

Namespace(data_dir='../data/eval/toxigen/', save_dir='../results/baselines/huggyllama/llama-7b//eval/toxigen/', model_name_or_path='../results/baselines/huggyllama/llama-7b/', tokenizer_name_or_path=None, use_slow_tokenizer=False, openai_engine=None, eval_batch_size=10, classifier_batch_size=32, classifier_device='cuda', load_in_8bit=False, gptq=False, use_chat_format=True, chat_formatting_function='eval.templates.create_prompt_with_tulu_chat_format', use_vllm=False, max_prompts_per_group=500, max_new_tokens=512)

In [15]:
random.seed(42)

all_prompts = []
prompt_files = glob.glob(os.path.join(args.data_dir, "*.txt"))
for task_file in tqdm(prompt_files, desc="Loading prompts"):
    with open(task_file, "r") as f:
        group_name = os.path.basename(task_file).split(".")[0]
        label = group_name.split("_")[0]
        minority_group = "_".join(group_name.split("_")[1:])
        group_prompts = [line.strip() for line in f]
        random.shuffle(group_prompts)
        group_prompts = group_prompts[:args.max_prompts_per_group]
        for prompt in group_prompts:
            # minor cleaning: replace \\ with \
            prompt = prompt.replace("\\\\", "\\")
            prompt = prompt.replace("\\n", "\n")
            all_prompts.append(
                {
                    "text": prompt,
                    "label": label,
                    "target_groups": [minority_group],
                }
            )

Loading prompts: 100%|██████████| 14/14 [00:00<00:00, 278.37it/s]

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [11]:
all(len(x['target_groups'])==1 for x in all_prompts)

True

In [12]:
l = [x['target_groups'][0] for x in all_prompts]
from collections import Counter

d = Counter(l)
d

Counter({'physical_disability': 500,
         'chinese': 500,
         'black': 500,
         'mexican': 500,
         'women': 500,
         'latino': 500,
         'native_american': 500,
         'lgbtq': 500,
         'muslim': 500,
         'mental_disability': 500,
         'trans': 500,
         'middle_east': 500,
         'jewish': 500,
         'asian': 500})

In [ ]:

# we assume running on a gpu here.
toxigenRobertaTokenizer = AutoTokenizer.from_pretrained("tomh/toxigen_roberta")
toxigenRobertaModel = AutoModelForSequenceClassification.from_pretrained(
    "tomh/toxigen_roberta"
).cuda()

os.makedirs(args.save_dir, exist_ok=True)
os.makedirs(os.path.join(args.save_dir, "predictions"), exist_ok=True)


In [ ]:

print("Loading model and tokenizer...")
model, tokenizer = load_hf_lm_and_tokenizer(
    model_name_or_path=args.model_name_or_path,
    tokenizer_name_or_path=args.tokenizer_name_or_path,
    load_in_8bit=args.load_in_8bit,
    device_map="balanced_low_0" if torch.cuda.device_count() > 1 else "auto",
    gptq_model=args.gptq,
    use_fast_tokenizer=not args.use_slow_tokenizer,
)

## eval_hf_model function 

In [ ]:

# results = eval_hf_model(
#     args,
#     model,
#     tokenizer,
#     all_prompts,
#     toxigenRobertaTokenizer,
#     toxigenRobertaModel,
#     args.classifier_batch_size,
#     save_path=os.path.join(args.save_dir, "predictions", "toxigen.jsonl"),
# )

examples = all_prompts
hate_classifier_tokenizer = toxigenRobertaTokenizer
hate_classifier_model = toxigenRobertaModel
classifer_batch_size = 32
save_path = os.path.join(args.save_dir, "predictions", "toxigen.jsonl")



In [ ]:

if save_path:
    fout = open(save_path, "w")

prompts = []
chat_formatting_function = dynamic_import_function(args.chat_formatting_function) if args.use_chat_format else None
for example in examples:
    if args.use_chat_format:
        messages = [{"role": "user", "content": "Complete the following: " + example["text"]}]
        prompt = chat_formatting_function(messages, add_bos=False)
    else:
        prompt = example["text"]
    prompts.append(prompt)



In [ ]:

# get the last token because the tokenizer may add space tokens at the start.
# wpq: t5 tokenizer strips `\n`. don't use `\n` as stop sequence. just generate to max length or encounters <\s>. 
new_line_token = tokenizer.encode("\n", add_special_tokens=False)
stop_id_sequences = [[new_line_token[-1]]] if new_line_token else None

# wpq: modify `max_new_tokens=512` to `256` for faster generation.
generation_kwargs = {'max_new_tokens': args.max_new_tokens}


outputs = generate_completions(
    model=model,
    tokenizer=tokenizer,
    prompts=prompts[:10],
    batch_size=args.eval_batch_size,
    stop_id_sequences=stop_id_sequences,
    **generation_kwargs,
)

In [ ]:


classifications = score_generations(
    hate_classifier_tokenizer,
    hate_classifier_model,
    outputs,
    batch_size=classifer_batch_size,
)

In [ ]:


all_groups_scores = defaultdict(list)
for example, output, classification in zip(examples, outputs, classifications):
    example["prediction"] = output
    example["is_toxic"] = classification
    for group in example["target_groups"]:
        all_groups_scores[group].append(classification)
    if save_path:
        fout.write(json.dumps(example) + "\n")

performance = {}
# calculate score by targeted group
for group in all_groups_scores:
    performance[group] = sum(all_groups_scores[group]) / len(
        all_groups_scores[group]
    )
# overall perf
performance["overall"] = sum(classifications) / len(classifications)
performance